In [1]:
import minsearch

In [2]:
import openai
import json

In [3]:
with open("documents.json", "rb") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course in docs_raw:
    for doc in course["documents"]:
        doc['course'] = course["course"]
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [7]:
q = "can i still enroll in the course after it has started"

In [8]:
index.fit(documents)

In [9]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    filter_dict={"course" : "data-engineering-zoomcamp"},
    query = q,
    boost_dict = boost,
    num_results = 5
)

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": q}]
)

In [13]:
response.choices[0].message.content

'Whether you can still enroll in a course after it has started largely depends on the specific policies of the institution offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Course Website or Syllabus**: Often, information about enrollment deadlines is provided on the course website or syllabus.\n\n2. **Contact the Instructor**: The course instructor may have the authority to allow late enrollments. Explain your situation and express your interest in the course.\n\n3. **Reach Out to the Registrar or Admissions Office**: These offices can provide you with the official policies regarding late enrollment and might assist you in the process.\n\n4. **Look for Add/Drop Period Information**: Many institutions have a designated add/drop period during which you can make changes to your course schedule without penalty.\n\n5. **Consider the Format of the Course**: For online courses, there might be more flexibility compared to in-person courses. Some online plat

In [15]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        filter_dict={"course" : "data-engineering-zoomcamp"},
        query = query,
        boost_dict = boost,
        num_results = 5
    )
    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
        
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [18]:
query = "how do i build kafka"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag("how do i get the certificate")

'To obtain a certificate for the course, you need to complete the course with a "live" cohort. Certificates are not awarded for completing the course in a self-paced mode because you are required to peer-review capstone projects, and this activity is only available when the course is running in sync with a live cohort.'

In [ ]:
documents[0]

In [20]:
from elasticsearch import Elasticsearch

In [21]:
es_client = Elasticsearch("http://localhost:9200")

In [22]:
es_client.info()

ObjectApiResponse({'name': '4f197f2c5f20', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'XwjKmAS3Thul6AK1xBAVUg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name = "course-questions"
#es_client.indices.create(index= index_name, body=index_settings)

In [27]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [31]:
query = "How can i get the course certificate"

In [41]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_source'])
    return result_docs

In [42]:
elastic_search(query)

[{'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.\nYou can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.',
  'section': 'General course-related questions',
  'question': 'Course - Can I get support if I take the course in the self-paced 

In [43]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
rag(query)

'To receive a certificate for the course, you must complete the course with a "live" cohort. Certificates are not awarded for completing the course in a self-paced mode because you need to peer-review capstone projects, which can only be done during the time the course is running.'